In [3]:
from __future__ import print_function, division
import sys
import copy
import os
import datetime
import time
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
import torchvision
import numpy as np
from torch.optim import lr_scheduler
import torch.optim as optim
import torch.nn as nn
import torch
import os
from src.shared import gen_train_val

plt.ion()   # interactive mode

In [4]:
# for compatibility with old code
is_local = True

In [5]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

dataset_name = "CUB_200"
# dataset_name = "CUB_200_2011/CUB_200_2011/images"
data_dir = f"datasets/{dataset_name}" if is_local else f"/users/tjiang12/data/tjiang12/{dataset_name}"
dataset = datasets.ImageFolder(data_dir, data_transforms['train'])

val_size = int(0.3 * len(dataset))
train_size = len(dataset) - val_size

train_and_val = torch.utils.data.random_split(dataset, [train_size, val_size])

image_datasets = dict(zip(['train', 'val'], train_and_val))
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = [classname.split('.')[1].replace('_', ' ') for classname in dataset.classes]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# path of weights transfered from CCV
PATH = "weights/test_pseudo"
# PATH = "weights/resnet50_CUB200_66pct"
model_loaded = models.resnet50()
num_ftrs = model_loaded.fc.in_features
model_loaded.fc = nn.Linear(num_ftrs, num_classes)

model_loaded = model_loaded.to(device)
model_loaded.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))

NameError: name 'num_classes' is not defined

In [1]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//3, 3, images_so_far)
                ax.axis('off')
                pred_status = "✔" if preds[j] == labels[j] else "✘, " + class_names[labels[j]]
                ax.set_title('{} ({})'.format(class_names[preds[j]], pred_status))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)


def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [2]:
visualize_model(model_loaded)

NameError: name 'model_loaded' is not defined

In [149]:
# Experiment: Evaluate trained model's performance on unlabeled data
model_loaded.eval()

with torch.no_grad():
    running_corrects = 0
    seen = 0
    for i, (inputs, labels) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)

        outputs = model_loaded(inputs)
        _, preds = torch.max(outputs, 1)
        print(preds)
        running_corrects += torch.sum(preds == labels.data)
        seen += len(inputs)
        print(running_corrects, "/", seen, running_corrects / seen)

tensor([ 27,  81, 125,  56])
tensor(4) / 4 tensor(1.)
tensor([ 21,   1, 106,  14])
tensor(6) / 8 tensor(0.7500)
tensor([160, 143,  90, 105])
tensor(8) / 12 tensor(0.6667)
tensor([97, 87, 52, 80])
tensor(10) / 16 tensor(0.6250)
tensor([179, 130,  83,  58])
tensor(14) / 20 tensor(0.7000)
tensor([157, 105,  75, 154])
tensor(18) / 24 tensor(0.7500)
tensor([ 92,  27, 197,  31])
tensor(20) / 28 tensor(0.7143)
tensor([112,   4,  39, 106])
tensor(22) / 32 tensor(0.6875)
tensor([110,  82,  51,  58])
tensor(25) / 36 tensor(0.6944)
tensor([153,  99, 119, 106])
tensor(26) / 40 tensor(0.6500)
tensor([ 80, 112,   6,  84])
tensor(30) / 44 tensor(0.6818)
tensor([126, 106,  35, 105])
tensor(33) / 48 tensor(0.6875)
tensor([ 37,  26, 165, 188])
tensor(36) / 52 tensor(0.6923)
tensor([136, 155,  91,  26])
tensor(40) / 56 tensor(0.7143)
tensor([135, 106, 161,  77])
tensor(43) / 60 tensor(0.7167)
tensor([ 93,  67, 137, 145])
tensor(46) / 64 tensor(0.7188)
tensor([ 84,  26, 168,  82])
tensor(50) / 68 tensor(0

KeyboardInterrupt: 

In [ ]:
# Load larger unlabeled set (CUB 200 2011)
unlabeled_data_dir = "datasets/CUB_200"
unlabeled_dataset = datasets.ImageFolder(
    unlabeled_data_dir, data_transforms['train'])
val_size = int(val_pct * len(dataset))
train_size = len(dataset) - val_size

train_and_val = torch.utils.data.random_split(dataset, [train_size, val_size])

unlabeled_image_datasets = dict(zip(['train', 'val'], train_and_val))
unlabeled_dataloaders = {x: torch.utils.data.DataLoader(unlabeled_image_datasets[x], batch_size=4,
                                              shuffle=True)
                         for x in ['train', 'val']}

In [ ]:
import random
model_loaded.eval()

running_corrects = 0
seen = 0
with torch.no_grad():
#     while True:
#         i = random.randint(0,len(unlabeled_dataset))
    for i, j in unlabeled_dataset:
#         print(i)
#         i, j = unlabeled_dataset.__getitem__(i)
#     for i, j in unlabeled_dataset:
    #     print(i.shape)
    #     print(i)
    # #     i = torch.reshape(i)
    #     print(i)
    #     print(i.shape)
        i = i.to(device)
        outputs = model_loaded(torch.reshape(i,(1,3,224,224)))
        _, preds = torch.max(outputs, 1)
        print(preds)
        print(j)
        running_corrects += torch.sum(preds == j)
        seen += 1
        print(running_corrects, "/", seen, running_corrects / seen)

In [132]:
# Experiment: Evaluate trained model's performance on unlabeled data
model_loaded.eval()

with torch.no_grad():
    running_corrects = 0
    seen = 0
    for i, (inputs, labels) in enumerate(unlabeled_dataloaders['val']):
        inputs = inputs.to(device)

#         outputs = model_loaded(inputs[0:1])
        print(inputs[0:1].shape)
        print(inputs[0:1][0].shape)
        reshaped = torch.reshape(inputs[0:1][0], (1,3,224,224))
        print(reshaped.shape)
        outputs = model_loaded(reshaped)
        
        _, preds = torch.max(outputs, 1)
        print(preds)
        running_corrects += torch.sum(preds == labels.data[0:1])
        seen += len(inputs[0:1])
        print(running_corrects, "/", seen, running_corrects / seen)

torch.Size([1, 3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([167])
tensor(0) / 1 tensor(0.)
torch.Size([1, 3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([31])
tensor(0) / 2 tensor(0.)
torch.Size([1, 3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([93])
tensor(1) / 3 tensor(0.3333)
torch.Size([1, 3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([136])
tensor(2) / 4 tensor(0.5000)
torch.Size([1, 3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([15])
tensor(3) / 5 tensor(0.6000)
torch.Size([1, 3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([109])
tensor(4) / 6 tensor(0.6667)
torch.Size([1, 3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([121])
tensor(5) / 7 tensor(0.7143)
torch.Size([1, 3, 224, 224])
torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
tensor([68])
tensor(6) / 8 tensor(0.75

KeyboardInterrupt: 

In [ ]:
# # Experiment: Evaluate trained model's performance on unlabeled data
# model_loaded.eval()

# with torch.no_grad():
#     running_corrects = 0
#     seen = 0
#     for i, (inputs, labels) in enumerate(dataloaders['val']):
#         inputs = inputs.to(device)

#         outputs = model_loaded(inputs)
#         _, preds = torch.max(outputs, 1)
#         print(preds)
#         running_corrects += torch.sum(preds == labels.data)
#         seen += len(inputs)
#         print(running_corrects, "/", seen, running_corrects / seen)

In [131]:
with torch.no_grad():
    running_corrects = 0
    seen = 0
    for i, (inputs, labels) in enumerate(dataset):
        inputs = inputs.to(device)

        outputs = model_loaded(torch.reshape(inputs, (1,3,224,224)))
        _, preds = torch.max(outputs, 1)
        print(preds)
        running_corrects += torch.sum(preds == labels)
        seen += 1
        print(running_corrects, "/", seen, running_corrects / seen)

tensor([141])
tensor(0) / 1 tensor(0.)
tensor([192])
tensor(0) / 2 tensor(0.)
tensor([141])
tensor(0) / 3 tensor(0.)
tensor([192])
tensor(0) / 4 tensor(0.)
tensor([141])
tensor(0) / 5 tensor(0.)
tensor([141])
tensor(0) / 6 tensor(0.)
tensor([141])
tensor(0) / 7 tensor(0.)
tensor([141])
tensor(0) / 8 tensor(0.)
tensor([141])
tensor(0) / 9 tensor(0.)


KeyboardInterrupt: 